# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 2:55PM,247783,10,CTF0011437,"Citieffe, Inc.",Released
1,Oct 3 2022 2:53PM,247782,20,8605591-T,"ACI Healthcare USA, Inc.",Released
2,Oct 3 2022 2:42PM,247780,19,CLI00392.1,"CLINUVEL, Inc.",Released
3,Oct 3 2022 2:23PM,247779,10,SOIVA0000534,Ivaoes Animal Health,Released
4,Oct 3 2022 2:23PM,247778,10,SOTAH0000801,"Nextsource Biotechnology, LLC",Released
5,Oct 3 2022 2:17PM,247777,10,8605590,Yusen – 3D Matrix,Released
6,Oct 3 2022 2:15PM,247776,16,8593021,Sartorius Lab Products and Service,Released
7,Oct 3 2022 2:15PM,247776,16,8593023,Sartorius Lab Products and Service,Released
8,Oct 3 2022 2:15PM,247776,16,8593563,Sartorius Lab Products and Service,Released
9,Oct 3 2022 2:15PM,247776,16,8593586,Sartorius Lab Products and Service,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,247778,Released,1
41,247779,Released,1
42,247780,Released,1
43,247782,Released,1
44,247783,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247778,NaN,NaN,1.0
247779,NaN,NaN,1.0
247780,NaN,NaN,1.0
247782,NaN,NaN,1.0
247783,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247684,0.0,0.0,15.0
247692,0.0,1.0,0.0
247706,0.0,0.0,1.0
247707,0.0,0.0,1.0
247708,1.0,3.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247684,0,0,15
247692,0,1,0
247706,0,0,1
247707,0,0,1
247708,1,3,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247684,0,0,15
1,247692,0,1,0
2,247706,0,0,1
3,247707,0,0,1
4,247708,1,3,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247684,,,15
1,247692,,1,
2,247706,,,1
3,247707,,,1
4,247708,1,3,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 2:55PM,247783,10,"Citieffe, Inc."
1,Oct 3 2022 2:53PM,247782,20,"ACI Healthcare USA, Inc."
2,Oct 3 2022 2:42PM,247780,19,"CLINUVEL, Inc."
3,Oct 3 2022 2:23PM,247779,10,Ivaoes Animal Health
4,Oct 3 2022 2:23PM,247778,10,"Nextsource Biotechnology, LLC"
5,Oct 3 2022 2:17PM,247777,10,Yusen – 3D Matrix
6,Oct 3 2022 2:15PM,247776,16,Sartorius Lab Products and Service
10,Oct 3 2022 2:12PM,247775,10,Yusen – 3D Matrix
11,Oct 3 2022 2:06PM,247774,10,Emerginnova
12,Oct 3 2022 2:04PM,247773,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 3 2022 2:55PM,247783,10,"Citieffe, Inc.",,,1
1,Oct 3 2022 2:53PM,247782,20,"ACI Healthcare USA, Inc.",,,1
2,Oct 3 2022 2:42PM,247780,19,"CLINUVEL, Inc.",,,1
3,Oct 3 2022 2:23PM,247779,10,Ivaoes Animal Health,,,1
4,Oct 3 2022 2:23PM,247778,10,"Nextsource Biotechnology, LLC",,,1
5,Oct 3 2022 2:17PM,247777,10,Yusen – 3D Matrix,,,1
6,Oct 3 2022 2:15PM,247776,16,Sartorius Lab Products and Service,,,4
7,Oct 3 2022 2:12PM,247775,10,Yusen – 3D Matrix,,,1
8,Oct 3 2022 2:06PM,247774,10,Emerginnova,,,1
9,Oct 3 2022 2:04PM,247773,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 2:55PM,247783,10,"Citieffe, Inc.",1,,
1,Oct 3 2022 2:53PM,247782,20,"ACI Healthcare USA, Inc.",1,,
2,Oct 3 2022 2:42PM,247780,19,"CLINUVEL, Inc.",1,,
3,Oct 3 2022 2:23PM,247779,10,Ivaoes Animal Health,1,,
4,Oct 3 2022 2:23PM,247778,10,"Nextsource Biotechnology, LLC",1,,
5,Oct 3 2022 2:17PM,247777,10,Yusen – 3D Matrix,1,,
6,Oct 3 2022 2:15PM,247776,16,Sartorius Lab Products and Service,4,,
7,Oct 3 2022 2:12PM,247775,10,Yusen – 3D Matrix,1,,
8,Oct 3 2022 2:06PM,247774,10,Emerginnova,1,,
9,Oct 3 2022 2:04PM,247773,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 2:55PM,247783,10,"Citieffe, Inc.",1,,
1,Oct 3 2022 2:53PM,247782,20,"ACI Healthcare USA, Inc.",1,,
2,Oct 3 2022 2:42PM,247780,19,"CLINUVEL, Inc.",1,,
3,Oct 3 2022 2:23PM,247779,10,Ivaoes Animal Health,1,,
4,Oct 3 2022 2:23PM,247778,10,"Nextsource Biotechnology, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 2:55PM,247783,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Oct 3 2022 2:53PM,247782,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Oct 3 2022 2:42PM,247780,19,"CLINUVEL, Inc.",1.0,NaN,NaN
3,Oct 3 2022 2:23PM,247779,10,Ivaoes Animal Health,1.0,NaN,NaN
4,Oct 3 2022 2:23PM,247778,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 2:55PM,247783,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Oct 3 2022 2:53PM,247782,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Oct 3 2022 2:42PM,247780,19,"CLINUVEL, Inc.",1.0,0.0,0.0
3,Oct 3 2022 2:23PM,247779,10,Ivaoes Animal Health,1.0,0.0,0.0
4,Oct 3 2022 2:23PM,247778,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5946114,244.0,9.0,1.0
15,743252,16.0,15.0,0.0
16,1238681,16.0,0.0,0.0
19,743293,13.0,13.0,0.0
20,495501,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5946114,244.0,9.0,1.0
1,15,743252,16.0,15.0,0.0
2,16,1238681,16.0,0.0,0.0
3,19,743293,13.0,13.0,0.0
4,20,495501,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,244.0,9.0,1.0
1,15,16.0,15.0,0.0
2,16,16.0,0.0,0.0
3,19,13.0,13.0,0.0
4,20,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,244.0
1,15,Released,16.0
2,16,Released,16.0
3,19,Released,13.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,9.0,15.0,0.0,13.0,0.0
Released,244.0,16.0,16.0,13.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,9.0,15.0,0.0,13.0,0.0
2,Released,244.0,16.0,16.0,13.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,9.0,15.0,0.0,13.0,0.0
2,Released,244.0,16.0,16.0,13.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()